# Vpogled v nepremičnine: Analiza slovenskega nepremičninskega trga

Ta projekt si prizadeva zagotoviti poglobljeno analizo slovenskega nepremičninskega trga, s poudarkom na prodaji in dolgoročnem najemu. Podatki za to analizo so bili pridobljeni s spletnega mesta https://www.nepremicnine.net/, ki zajema širok spekter nepremičninskih oglasov.

Z uporabo knjižnice Pandas v programskem jeziku Python znotraj okolja Jupyter Notebook bo projekt raziskal ključne tržne trende, cenovne vzorce in geografsko razporeditev nepremičnin. S pomočjo vizualizacije teh podatkov in izvedbe statističnih analiz bo projekt poskušal ponuditi dragocene vpoglede v dinamiko slovenskega nepremičninskega trga, kar bi lahko pomagalo kupcem, najemnikom in vlagateljem pri sprejemanju premišljenih odločitev.

# Vsebina
1. [Pregled trga](#Poglavje-1)
   - [Razporeditev oglasov po tipih (prodaja/oddaja)](#Razdelek-1.1)
   - [Razporeditev oglasov po tipu nepremičnine (hiša/stanovanje)](#Razdelek-1.2)
   - [Geografska razporeditev](#Razdelek-1.3)
2. [Analiza cen](#Poglavje-2)
   - [Razporeditev cen](#Razdelek-2.1)
     - [Razporeditev cen (prodaja)](#Podrazdelek-2.1.1)
     - [Razporeditev cen (oddaja)](#Podrazdelek-2.1.2)
   - [Povprečne in mediane cene](#Razdelek-2.2)
     - [Povprečne in mediane cene (prodaja)](#Podrazdelek-2.2.1)
     - [Povprečne in mediane cene (oddaja)](#Podrazdelek-2.2.2)
   - [Regionalne razlike v cenah](#Razdelek-2.3)
     - [Regionalne razlike v cenah (prodaja)](#Podrazdelek-2.3.1)
     - [Regionalne razlike v cenah (oddaja)](#Podrazdelek-2.3.2)
3. [Analiza značilnosti nepremičnin](#Poglavje-3)
   - [Leto gradnje](#Razdelek-3.1)
   - [Bivalni prostor](#Razdelek-3.2)
4. [Primerjalna analiza najema in prodaje](#Poglavje-4)
   - [Razmerje cen](#Razdelek-4.1)
   - [Donosnost najema](#Razdelek-4.2)
5. [Rezultati](#Poglavje-5)
     

### Uvoz podatkov in prilagoditev

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sbn
import matplotlib.pyplot as plt

df = pd.read_csv('data/ads.csv', index_col='ad_id')
sell_df = df[df['ad_type'] == 'prodaja']
rent_df = df[df['ad_type'] == 'oddaja']
pd.options.display.max_rows = 20
pd.options.mode.chained_assignment = None

In [ ]:
df.info()
df.head()

***
# Poglavje 1
# Pregled trga

Ta razdelek zagotavlja pregled trga nepremičnin z analizo razporeditve oglasov po tipu (prodaja ali oddaja), tipu nepremičnine (hiša ali stanovanje) in geografski lokaciji. Razumevanje teh razporeditev pomaga pri prepoznavanju ključnih tržnih trendov in relativne razpoložljivosti različnih vrst nepremičnin v različnih regijah.

## Razdelek 1.1
## Razporeditev oglasov po tipih (prodaja/oddaja)

Najprej si oglejmo razmerje med oglasi za prodajo in oddajo. Podatke bomo vizualizirali, da bomo videli, kako je trg razdeljen med tema dvema kategorijama.

In [ ]:
df['ad_type'].value_counts().plot.pie(
    title='Razporeditev oglasov po tipih (prodaja/oddaja)',
    labels=['Prodaja', 'Oddaja'],
    autopct='%.2f%%',
    shadow=True,        
    ylabel='',    
)

In [ ]:
df['ad_type'].value_counts()

## Razdelek 1.2
## Razporeditev oglasov po tipu nepremičnine (hiša/stanovanje)

Zdaj si oglejmo razporeditev oglasov med hišami in stanovanji. Ta analiza nam bo pomagala razumeti relativno razpoložljivost različnih vrst nepremičnin na trgu in ali obstaja prednost enega tipa pred drugim.

In [ ]:
df.groupby('real_estate_type').size().plot.pie(
    title='Razporeditev oglasov po tipu nepremičnine (hiša/stanovanje)',
    autopct='%.2f%%',
    shadow=True,        
    ylabel='',    
)

In [ ]:
df['real_estate_type'].value_counts()

Oglejmo si tudi razporeditev oglasov po tipu oglasa (prodaja/oddaja) in tipu nepremičnine (hiša/stanovanje) hkrati.

In [ ]:
df[['ad_type', 'real_estate_type']].value_counts().plot.pie(
    title='Razporeditev oglasov po tipu oglasa (prodaja/oddaja) in tipu nepremičnine (hiša/stanovanje)',
    autopct='%.2f%%',
    shadow=True,        
    ylabel='',    
)

In [ ]:
df[['ad_type', 'real_estate_type']].value_counts()

## Razdelek 1.3
## Geografska razporeditev

Zdaj bomo prikazali razporeditev nepremičninskih oglasov po različnih regijah. Ta geografska analiza bo razkrila območja z visoko koncentracijo oglasov, kar lahko kaže na vroče točke trga ali razlike v razpoložljivosti nepremičnin.

In [ ]:
df['region'].value_counts().plot.barh(
    title="Geografska razporeditev"
)

In [ ]:
df['region'].value_counts()

## Sklep

Po kratkem pregledu trga lahko zaključimo, da na njem prevladujejo oglasi za prodajo nepremičnin. Pri tem je razdelitev med prodajo stanovanj in hiš enaka, medtem ko se veliko več stanovanj oddaja v najem.

Če pogledamo razporeditev po regijah, je vidno, da je največ ponudb v Ljubljani.

***
# Poglavje 2
# Analiza cen

V tem razdelku bomo analizirali razporeditev in trende cen nepremičnin. To vključuje preučevanje splošne razporeditve cen, izračun povprečnih in medianih cen za različne tipe nepremičnin ter raziskovanje, kako se cene razlikujejo glede na regijo. Prav tako bomo raziskali razmerje med ceno in značilnostmi nepremičnin, kot so površina in število sob.

Posebej si bomo ogledali tudi specifične nepremičnine: samostojne hiše in 2-3 sobna stanovanja za prodajo, saj so po skromnem mnenju avtorja to najboljše nepremičnine za nakup, ter sobe, garsonjere in enosobna stanovanja za najem, saj so najbolj primerna za najem študentom.

## Razdelek 2.1
## Razporeditev cen

Najprej bomo preučili splošno razporeditev cen nepremičnin. To vključuje vizualizacijo razporeditve cen na trgu in razumevanje splošnega cenovnega razpona za prodajo in oddajo. Ta analiza zagotavlja osnovo za globlje vpoglede v cenovne trende.

### Podrazdelek 2.1.1
### Razporeditev cen (prodaja)

Za začetek si oglejmo 5 najdražjih in 5 najcenejših ponudb:

In [ ]:
sell_df.sort_values('price', ascending=False).head(5)

In [ ]:
sell_df.sort_values('price', ascending=True).head(5)

Izračunajmo razliko med najdražjim in najcenejšim oglasom:

In [ ]:
print(f'Razlika je {float(sell_df['price'].max() - sell_df['price'].min())} evrov.')

Oglejmo si najdražje in najcenejše samostojne hiše ter 2-3 sobna stanovanja.

**Samostojne hiše:**

In [ ]:
single_homes = sell_df[(sell_df['real_estate_type'] == 'hiša') & (sell_df['type'] == 'samostojna')]
single_homes.sort_values('price', ascending=False).head(5)

In [ ]:
single_homes.sort_values('price', ascending=True).head(5)

**2-sobna stanovanja:**

In [ ]:
two_room_apartments = sell_df[(sell_df['real_estate_type'] == 'stanovanje') & ((sell_df['type'] == '2-sobno') | (sell_df['type'] == '2,5-sobno'))]
two_room_apartments.sort_values('price', ascending=False).head(5)

In [ ]:
two_room_apartments.sort_values('price', ascending=True).head(5)

**3-sobna stanovanja:**

In [ ]:
three_room_apartments = sell_df[(sell_df['real_estate_type'] == 'stanovanje') & (sell_df['type'] == '3-sobno')]
three_room_apartments.sort_values('price', ascending=False).head(5)

In [ ]:
three_room_apartments.sort_values('price', ascending=True).head(5)

Oglejmo si splošno razporeditev cen.

In [ ]:
sell_df['price'].plot.hist(
    title='Razporeditev cen',
    bins=25, 
    xlim=[sell_df['price'].min(), sell_df['price'].max()]
)

Opazimo, da večina nepremičnin, ki so naprodaj, stane do 500.000 evrov. Za boljše razumevanje razporeditve cen bomo oglase razdelili v tri cenovne kategorije: od 0 do 500.000 evrov, od 500.000 do 1.000.000 evrov in od 1.000.000 evrov naprej.

In [ ]:
first_group = sell_df[sell_df['price'] < 500000]
first_group['price'].plot.hist(
    title='Razporeditev cen. Oglasi od 0 do 500.000 evrov',
    bins=20, 
    xlim=[first_group['price'].min(), first_group['price'].max()]
)

In [ ]:
second_group = sell_df[(sell_df['price'] >= 500000) & (sell_df['price'] < 1000000)]
second_group['price'].plot.hist(
    title='Razporeditev cen. Oglasi od 500.000 do 1.000.000 evrov',
    bins=20, 
    xlim=[second_group['price'].min(), second_group['price'].max()]
)

In [ ]:
third_group = sell_df[sell_df['price'] > 1000000]
third_group['price'].plot.hist(
    title='Razporeditev cen. Oglasi od 1.000.000 evrov naprej',
    bins=10, 
    xlim=[third_group['price'].min(), third_group['price'].max()]
)

### Podrazdelek 2.1.2
### Razporeditev cen (oddaja)

Spet si oglejmo 5 najdražjih in 5 najcenejših ponudb.

In [ ]:
rent_df.sort_values('price', ascending=False).head(5)

In [ ]:
rent_df.sort_values('price', ascending=True).head(5)

Izračunajmo razliko med najdražjim in najcenejšim oglasom.

In [ ]:
print(f'Razlika je {float(rent_df['price'].max() - rent_df['price'].min())} evrov.')

Oglejmo si najdražje in najcenejše sobe, garsonjere in enosobna stanovanja.

**Sobe:**

In [ ]:
rooms = rent_df[(rent_df['real_estate_type'] == 'stanovanje') & (rent_df['type'] == 'soba')]
rooms.sort_values('price', ascending=False).head(5)

In [ ]:
rooms.sort_values('price', ascending=True).head(5)

**Garsonjere:**

In [ ]:
studios = rent_df[(rent_df['real_estate_type'] == 'stanovanje') & (rent_df['type'] == 'garsonjera')]
studios.sort_values('price', ascending=False).head(5)

In [ ]:
studios.sort_values('price', ascending=True).head(5)

**1-sobna stanovanja:**

In [ ]:
one_room_apartments = rent_df[(rent_df['real_estate_type'] == 'stanovanje') & (rent_df['type'] == '1-sobno')]
one_room_apartments.sort_values('price', ascending=False).head(5)

In [ ]:
one_room_apartments.sort_values('price', ascending=True).head(5)

Oglejmo si splošno razporeditev cen.

In [ ]:
rent_df['price'].plot.hist(
    title='Razporeditev cen',
    bins=20, 
    xlim=[rent_df['price'].min(), rent_df['price'].max()]
)

Opazimo, da je večina oglasov za najem stanovanj v cenovnem razponu od 0 do 1000 evrov/mesec, zato bomo podrobneje pogledali razporeditev oglasov v tem cenovnem razponu.

In [ ]:
first_group = rent_df[rent_df['price'] <= 1000]
first_group['price'].plot.hist(
    title='Razporeditev cen. Oglasi od 0 do 1000 evrov',
    bins=20, 
    xlim=[first_group['price'].min(), first_group['price'].max()]
)

## Razdelek 2.2
##  Povprečne in mediane cene

V tem podrazdelku bomo izračunali in primerjali povprečne in mediane cene nepremičnin. Analiza bo segmentirana po tipu nepremičnine (hiše, stanovanja) in tipu oglasa (prodaja, najem), da bi izpostavili morebitne razlike. Razumevanje teh mer centralne tendence pomaga oceniti tipične tržne cene.

### Podrazdelek 2.2.1
### Povprečne in mediane cene (prodaja)

Izračunajmo povprečne in mediane vrednosti cene prodaje hiš in stanovanj.

In [ ]:
sell_df.groupby('real_estate_type')['price'].agg(['mean', 'median'])

Posebej bomo pregledali povprečne in mediane cene prodaje samostojnih hiš in 2-3 sobnih stanovanj.

**Samostojne hiše:**

In [ ]:
single_homes['price'].agg(['mean', 'median'])

**2-sobni stanovanja:**

In [ ]:
two_room_apartments['price'].agg(['mean', 'median'])

**3-sobni stanovanja:**

In [ ]:
three_room_apartments['price'].agg(['mean', 'median'])

### Podrazdelek 2.2.2
### Povprečne in mediane cene (oddaja)

Izračunajmo povprečne in mediane vrednosti cene najema hiš in stanovanj.

In [ ]:
rent_df.groupby('real_estate_type')['price'].agg(['mean', 'median'])

Posebej bomo pregledali povprečne in mediane cene najema sob, garsonjer in 1 sobnih stanovanj.

**Sobe:**

In [ ]:
rooms['price'].agg(['mean', 'median'])

**Garsonjere:**

In [ ]:
studios['price'].agg(['mean', 'median'])

**1-sobni stanovanja:**

In [ ]:
one_room_apartments['price'].agg(['mean', 'median'])

## Razdelek 2.3
## Regionalne razlike v cenah

Tukaj bomo raziskali, kako se cene nepremičnin razlikujejo po različnih regijah in mestih. S primerjavo povprečnih cen na različnih lokacijah lahko prepoznamo dražja in cenejša območja, kar nam omogoča vpogled v regionalno dinamiko cen.

### Podrazdelek 2.3.1
### Regionalne razlike v cenah (prodaja)

Oglejmo si, kako se povprečne in mediane cene spreminjajo glede na regijo:

In [ ]:
sell_df.groupby('region')['price'].agg(['mean', 'median']).sort_values(['mean', 'median'], ascending=[False, False])

In [ ]:
sell_df.groupby('region')['price'].agg(['mean', 'median']).sort_values(['mean', 'median'], ascending=[False, False]).plot.barh(
    title='Regionalne razlike v cenah'
)

Vidimo, da so najdražje nepremičnine v Ljubljani, njene okolice in na morju.

Zdaj si bomo ogledali 10 najdražjih mest za nakup nepremičnin, v katerih je objavljenih vsaj 25 oglasov.

In [ ]:
cities_25 = []
for city in set(sell_df['city']):
    if int(sell_df['city'].value_counts()[city]) >= 25:
        cities_25.append(city)

In [ ]:
sell_df[sell_df['city'].isin(cities_25)].groupby('city')['price'].agg(['mean', 'median']).sort_values(['mean', 'median'], ascending=[False, False]).head(10)

In [ ]:
sell_df[sell_df['city'].isin(cities_25)].groupby('city')['price'].agg(['mean', 'median']).sort_values(['mean', 'median'], ascending=[False, False]).head(10).plot.barh(
    title='Mestne razlike v cenah (mesta z vsaj 25 oglasov)'
)

In enako bomo storili z mesti, v katerih je objavljenih več kot 100 oglasov.

In [ ]:
cities_100 = []
for city in set(sell_df['city']):
    if int(sell_df['city'].value_counts()[city]) >= 100:
        cities_100.append(city)

In [ ]:
sell_df[sell_df['city'].isin(cities_100)].groupby('city')['price'].agg(['mean', 'median']).sort_values(['mean', 'median'], ascending=[False, False]).head(10)

In [ ]:
sell_df[sell_df['city'].isin(cities_100)].groupby('city')['price'].agg(['mean', 'median']).sort_values(['mean', 'median'], ascending=[False, False]).head(10).plot.barh(
    title='Mestne razlike v cenah (mesta z več kot 100 oglasov)'
)

### Podrazdelek 2.3.2
### Regionalne razlike v cenah (oddaja)

Oglejmo si, kako se povprečne in mediane cene spreminjajo glede na regijo:

In [ ]:
rent_df.groupby('region')['price'].agg(['mean', 'median']).sort_values(['mean', 'median'], ascending=[False, False])

In [ ]:
rent_df.groupby('region')['price'].agg(['mean', 'median']).sort_values(['mean', 'median'], ascending=[False, False]).plot.barh(
    title='Regionalne razlike v cenah'
)

Spet vidimo, da so najdražje nepremičnine za najem v Ljubljani, njene okolice in na morju.

Zdaj si bomo ogledali 10 najdražjih mest za najem nepremičnin, v katerih je objavljenih vsaj 10 oglasov.

In [ ]:
cities_25_rent = []
for city in set(rent_df['city']):
    if int(rent_df['city'].value_counts()[city]) >= 10:
        cities_25_rent.append(city)

In [ ]:
rent_df[rent_df['city'].isin(cities_25_rent)].groupby('city')['price'].agg(['mean', 'median']).sort_values(['mean', 'median'], ascending=[False, False]).head(10)

In [ ]:
rent_df[rent_df['city'].isin(cities_25_rent)].groupby('city')['price'].agg(['mean', 'median']).sort_values(['mean', 'median'], ascending=[False, False]).head(10).plot.barh(
    title='Mestne razlike v cenah (mesta z vsaj 10 oglasov)'
)

In enako bomo storili z mesti, v katerih je objavljenih več kot 25 oglasov.

In [ ]:
cities_100_rent = []
for city in set(rent_df['city']):
    if int(rent_df['city'].value_counts()[city]) >= 25:
        cities_100_rent.append(city)

In [ ]:
rent_df[rent_df['city'].isin(cities_100_rent)].groupby('city')['price'].agg(['mean', 'median']).sort_values(['mean', 'median'], ascending=[False, False]).head(10)

In [ ]:
rent_df[rent_df['city'].isin(cities_100_rent)].groupby('city')['price'].agg(['mean', 'median']).sort_values(['mean', 'median'], ascending=[False, False]).head(10).plot.barh(
    title='Mestne razlike v cenah (mesta z več kot 25 oglasov)'
)

***
# Poglavje 3
# Analiza značilnosti nepremičnin

Ta razdelek se osredotoča na analizo specifičnih značilnosti nepremičnin, kot so leto gradnje in bivalni prostor. Raziskali bomo, kako ti dejavniki vplivajo na cene nepremičnin in druge tržne dinamike. Razumevanje teh značilnosti lahko zagotovi globlje vpoglede v to, kaj vpliva na vrednost nepremičnine.

Na začetku poglavja si bomo ogledali, kako so vse zanimive značilnosti (region, mesto, bivalni prostor, površina zemljišča, leto gradnje in cena) povezane med seboj. Za to bomo vse zanimive podatke tipa `string` pretvorili v številčno obliko (regije in mesta bomo ocenili glede na število oglasov v njih) in narisali korelacijsko matriko.

Omejili se bomo samo na oglase za prodajo.

In [ ]:
def interpreter(data_frame, column_index, transitional_dict):

    for i_id, object_ in enumerate(data_frame.iloc [:, column_index]):
        numeric_df.iat[i_id, column_index] = transitional_dict[object_]

In [ ]:
transitional_dict_regions = {}
for region in set(sell_df['region']):
    points = int(sell_df.groupby('region').size()[region])
    transitional_dict_regions[region] = points

In [ ]:
transitional_dict_cities = {}
for city in set(sell_df['city']):
    points = int(sell_df.groupby('city').size()[city])
    transitional_dict_cities[city] = points

In [ ]:
numeric_df = sell_df.copy()

interpreter(numeric_df, 2, transitional_dict_regions)
numeric_df['region'] = numeric_df['region'].astype(int)

interpreter(numeric_df, 3, transitional_dict_cities)
numeric_df['city'] = numeric_df['city'].astype(int)

In [ ]:
numeric_df = numeric_df.select_dtypes(include=np.number)
matrix = numeric_df.corr()
sbn.heatmap(matrix, annot=True, cmap="coolwarm", vmin=-1, vmax=1)
plt.xticks(rotation=45, ha="right")
plt.title("Korelacijska matrika")

Vidimo, da na ceno najbolj vplivata regija in mesto. Vpliv bivalnega prostora in leta gradnje se zdi zanemarljiv. Da bi ugotovili, ali je to res tako, bomo preučili, kako te značilnosti nepremičnine vplivajo na njeno vrednost v posameznih regijah.

## Razdelek 3.1
## Leto gradnje

V tem podrazdelku bomo analizirali razporeditev nepremičnin glede na leto gradnje. Prav tako bomo primerjali, kako starost nepremičnine vpliva na njeno ceno in druge značilnosti. Ta analiza lahko razkrije trende, povezane s povpraševanjem po starejših ali novejših nepremičninah.

Za začetek si bomo ogledali razporeditev števila oglasov glede na leto gradnje.

In [ ]:
sell_df['building_year'].plot.hist(
    title='Razporeditev števila oglasov glede na leto gradnje',
    bins=20
)

Vidimo, da so na trgu v prevladujočem številu predstavljeni objekti, zgrajeni po letu 1900. Zato si bomo ogledali, kako leto gradnje za objekte, zgrajene po letu 1900, vpliva na njihovo ceno v regijah, kot sta Ljubljana, Podravska in Južna Primorska.

In [ ]:
for index, region in enumerate(['ljubljana mesto', 'podravska']):
    tmp_df = sell_df[(sell_df['region'] == region) & ((sell_df['building_year'] >= 1900))]
    
    plt.subplot (2, 1, index + 1)
    plt.scatter(
        x=tmp_df['building_year'], 
        y=tmp_df['price']
    )
    plt.title(region)
    plt.subplots_adjust(hspace=0.5)

In [ ]:
for index, region in enumerate(['ljubljana mesto', 'podravska']):
    tmp_df = sell_df[(sell_df['region'] == region) & ((sell_df['building_year'] >= 1900))]
    
    plt.subplot (1, 2, index + 1)
    matrix = tmp_df[['building_year', 'price']].corr()
    sbn.heatmap(matrix, annot=True, cmap="coolwarm", vmin=-1, vmax=1)
    plt.xticks(rotation=45, ha="right")
    plt.title(region)
    plt.subplots_adjust(wspace=0.5)

Vidimo, da se potrjuje malo vpliva leta gradnje na vrednost nepremičnine.

## Razdelek 3.2
## Bivalni prostor

Ta podrazdelek bo raziskal razmerje med bivalno površino in ceno nepremičnine. Analizirali bomo, kako ta dejavnik vpliva na vrednost nepremičnine.

In [ ]:
for index, region in enumerate(['ljubljana mesto', 'podravska']):
    tmp_df = sell_df[(sell_df['region'] == region)]
    
    plt.subplot (2, 1, index + 1)
    plt.scatter(
        x=tmp_df['living_area'], 
        y=tmp_df['price']
    )
    plt.title(region)
    plt.subplots_adjust(hspace=0.5)

In [ ]:
for index, region in enumerate(['ljubljana mesto', 'podravska']):
    tmp_df = sell_df[(sell_df['region'] == region) & ((sell_df['building_year'] >= 1900))]
    
    plt.subplot (1, 2, index + 1)
    matrix = tmp_df[['living_area', 'price']].corr()
    sbn.heatmap(matrix, annot=True, cmap="coolwarm", vmin=-1, vmax=1)
    plt.xticks(rotation=45, ha="right")
    plt.title(region)
    plt.subplots_adjust(wspace=0.5)

Vidimo, da vpliv bivalnega prostora na ceno nepremičnine v vsake regiji posebej postane precej močan. Z rastjo bivalnega prostora očitno narašča tudi vrednost nepremičnine.

***
# Poglavje 4
# Primerjalna analiza najema in prodaje

Tukaj bomo primerjali trge najema in prodaje, pri čemer se bomo osredotočili na razmerja cen in donosnost najema. Z analizo cene na kvadratni meter tako za najem kot za prodajo ter izračunom ocenjene donosnosti najema bomo lahko ponudili vpoglede v relativno donosnost najema v primerjavi z nakupom v različnih regijah in za različne tipe nepremičnin.

## Razdelek 4.1
## Razmerje cen

Primerjali bomo ceno na kvadratni meter za nepremičnine, oglaševane za prodajo, v primerjavi z najemom. Ta analiza bo razkrila relativno stroškovno učinkovitost najema v primerjavi z nakupom, kar nam bo omogočilo vpogled v to, katera možnost je finančno ugodnejša v različnih regijah.

Izračunajmo povprečno ceno na kvadratni meter za vsak oglas:

In [ ]:
df['price_per_sq_m'] = df['price'] / df['living_area']
sell_df = df[df['ad_type'] == 'prodaja']
rent_df = df[df['ad_type'] == 'oddaja']

Oglejmo si, v katerih regijah je najvišja cena na kvadratni meter.

In [ ]:
sell_df.groupby(['ad_type', 'region'])['price_per_sq_m'].agg(['mean']).sort_values('mean', ascending=False)

In [ ]:
rent_df.groupby(['ad_type', 'region'])['price_per_sq_m'].agg(['mean']).sort_values('mean', ascending=False)

Vidimo, da so najdražji regiji za nakup in najem nepremičnin: Ljubljana, Južna Primorska in Gorenjska. Najcenejše nepremičnine za nakup se nahajajo v takih regijah kot: Zasavska, Pomurska in Koroška, medtem ko so najcenejše nepremičnine za najem se nahajajo v takih regijah kot: Notranjska, Koroška in Zasavska.

Izračunajmo povprečno razmerje med ceno na kvadratni meter za prodajo in ceno na kvadratni meter za najem za vsako regijo.

In [ ]:
sale_to_rent_ratio = {}

for region in set(df['region']):
    tmp_df = df[df['region'] == region]
    sell_price = tmp_df[tmp_df['ad_type'] == 'prodaja']['price_per_sq_m'].mean()
    rent_price = tmp_df[tmp_df['ad_type'] == 'oddaja']['price_per_sq_m'].mean()
    ratio = float(sell_price / rent_price)
    sale_to_rent_ratio[region] = ratio

sale_to_rent_ratio_df = pd.DataFrame(list(sale_to_rent_ratio.items()), columns=['region', 'sale_to_rent_ratio'])
sale_to_rent_ratio_df.sort_values('sale_to_rent_ratio')
    

In [ ]:
sale_to_rent_ratio_df.sort_values('sale_to_rent_ratio', ascending=False).plot.barh(
    x='region'
)

## Razdelek 4.2
## Donosnost najema

V tem podrazdelku bomo izračunali ocenjeno donosnost najema, ki je razmerje med povprečno ceno najema in ceno prodaje. Ta kazalnik pomaga oceniti potencialni donos naložbe v nepremičnine za najem, kar zagotavlja vpogled za investitorje.

Bomo pogledali donosnost oddajanja v najem v odstotkih na leto garsonjer, 1-3 sobnih stanovanj ter samostojnih hiš v različnih regijah.

In [ ]:
def get_rental_income(real_estate_type, df):
    profit_by_region = {}

    for region in set(df['region']):
        tmp_df = df[(df['region'] == region) & (df['type'] == real_estate_type)]
        sell_price = tmp_df[tmp_df['ad_type'] == 'prodaja']['price'].mean()
        rent_price = tmp_df[tmp_df['ad_type'] == 'oddaja']['price'].mean()
        profit = float((rent_price * 12) / sell_price) * 100
        profit_by_region[region] = profit

    return profit_by_region

**Samostojne hiše:**

In [ ]:
profit_single_houses = get_rental_income('samostojna', df)

profit_single_houses_df = pd.DataFrame(list(profit_single_houses.items()), columns=['region', 'profit'])
profit_single_houses_df.sort_values('profit', ascending=False)

**Garsonjere:**

In [ ]:
profit_studios = get_rental_income('garsonjera', df)

profit_studios = pd.DataFrame(list(profit_studios.items()), columns=['region', 'profit'])
profit_studios.sort_values('profit', ascending=False)

**1-sobna stanovanja:**

In [ ]:
profit_one_room_apartments = get_rental_income('1-sobno', df)

profit_one_room_apartments = pd.DataFrame(list(profit_one_room_apartments.items()), columns=['region', 'profit'])
profit_one_room_apartments.sort_values('profit', ascending=False)

**2-sobna stanovanja:**

In [ ]:
profit_two_room_apartments = get_rental_income('2-sobno', df)

profit_two_room_apartments = pd.DataFrame(list(profit_two_room_apartments.items()), columns=['region', 'profit'])
profit_two_room_apartments.sort_values('profit', ascending=False)

**3-sobna stanovanja:**

In [ ]:
profit_three_room_apartments = get_rental_income('3-sobno', df)

profit_three_room_apartments = pd.DataFrame(list(profit_three_room_apartments.items()), columns=['region', 'profit'])
profit_three_room_apartments.sort_values('profit', ascending=False)

Izračunajmo tudi povprečno donosnost oddajanja v najem v Sloveniji kot celoti:

In [ ]:
mean_sell_price = df[df['ad_type'] == 'prodaja']['price'].mean()
mean_rent_price = df[df['ad_type'] == 'oddaja']['price'].mean()
print('Povprečna donosnost je {0:.2f}% na leto.'.format((mean_rent_price * 12) / mean_sell_price * 100))

***
# Poglavje 5
# Rezultati

Izpostavimo najbolj zanimive rezultate analize:

- Na trgu prevladujejo prodajni oglasi (6-kratna premoč nad oglasi za najem).
- Najem hiše bo težaven (le 140 ponudb).
- Ponudbe za prodajo se začnejo pri 15.000 evrih in segajo do 4.000.000 evrov.
- Povprečna cena 2-sobnega stanovanja znaša 200.000 evrov, medtem ko 3-sobno stanovanje stane 300.000 evrov.
- Študent lahko računa na sobo za 250 evrov na mesec ali na garsonjero/1-sobno stanovanje za 600 evrov na mesec.
- Najdražje nepremičnine so v Ljubljani in ob morju. Najdražje mesto za nakup nepremičnine je Portorož.
- Leto gradnje nima vpliva na ceno stanovanja. Regija ima na ceno stanovanja majhen vpliv.
- Donosnost oddajanja nepremičnin v najem se giblje med 3 in 7 odstotki.